In [9]:
import os
import cv2
import mediapipe as mp
import HandTrackingModule as HTM 
import numpy as np
import matplotlib as plt
import pandas as pd
%matplotlib inline


In [14]:
DATADIR = "dataset2"
def get_paths(DATADIR = "dataset2"):
    CATEGORIES = ["correct", "wrong_placement","totally_wrong"]
    video_dirs = {"correct":[],"wrong_placement":[],"totally_wrong":[]}
    for category in CATEGORIES:  
        path = os.path.join(DATADIR,category)
        for file in os.listdir(path):
            if(category == "correct"):
                video_dirs['correct'].append(path + '\\' + file)
            elif(category == "wrong_placement"):
                video_dirs['wrong_placement'].append(path + '\\' + file)
            elif(category == "totally_wrong"):
                video_dirs['totally_wrong'].append(path + '\\' + file)

    return video_dirs

def format_string(s):
    #x = s[1]
    #y = s[2]
    #id= s[0]
    #return {'id':id,'x':x,'y':y}
    return str(s).replace(',','').replace('[','').replace(']','')

def mirror_this(image_file, gray_scale=False, with_plot=False):
    
    image_mirror = np.fliplr(image_file)
    return image_mirror.astype(np.uint8).copy() 


video_dirs = get_paths()

In [15]:
def generate_cs_str(id,frame,screen_width,screen_height,coordinates_left,coordinates_right,label):
    """
    generates csv string from the given arguments
    Note:
    To get the exact number of commas in the string u need calculate (no of elements -1)
    Arguments:
    id:id of the video
    frame: frame number
    screen_width: width of the image 
    screen_height: height of the image
    coordinates_left : list of x,y,z normalized coordinates for left hand
    coordinates_right : list of x,y,z normalized coordinates for right hand
    label : label of the movement correct, incorrect ,incorrect movements and notes
    
    Returns:
    cs_string -- comma separated string
    """
    no_commas = 130
    comma_count = 4
    cs_str = f"{id},{frame},{screen_width},{screen_height}"
    if(len(coordinates_left) == 0):    
        #fill in 63 commas (3*21) 
        for i in range(63):
            cs_str += ','
            comma_count += 1
    else:
        for coordinate in coordinates_left:
            cs_str+= f',{coordinate[1]}'
            cs_str+= f',{coordinate[2]}'
            cs_str+= f',{coordinate[3]}'
            comma_count += 3
    if(len(coordinates_right) == 0):
        #fill 63 commas (3*21)
        for i in range(63):
            cs_str += ','
            comma_count += 1
    else:    
        for coordinate in coordinates_right:
            cs_str+= f',{coordinate[1]}'
            cs_str+= f',{coordinate[2]}'
            cs_str+= f',{coordinate[3]}'
            comma_count += 3

    
    while(comma_count != no_commas):
        cs_str += ','
        comma_count += 1
    cs_str += f',{label}'

    return cs_str

In [18]:
def extract_frames(write,dirName,fileName,id,movement):
    """
    This function splits each video into multiple frames 
    and generates a csv file of the hand and finger positions 
    
    Arguments:
    write -- boolean indicating to overwrite the image
    dirName -- directory to find the videos
    fileName -- Name of the video you want to be extracted
    id -- ith video being processed
    movement -- movement class
    
    output: None
    """
    drawingModule = mp.solutions.drawing_utils
    handsModule = mp.solutions.hands
    capture = cv2.VideoCapture(fileName)
    detector = HTM.handDetector(max_hands=2)
    frameNr = 0
    while (True):
        #overwrite drawn image draw is set true by default
        success, frame = capture.read()
        #stopping condition
        if not success:
            break
        img = frame
        img =  mirror_this(frame)
        img = detector.find_hands(img)

        hands_dict = detector.find_position2(img)
        coordinates_left = []
        coordinates_right = []
        
        screen_width = None
        screen_height = None
        #if hands are not detected dictionary is null which triggers an error
        if(hands_dict != None):
            #store left and right hand into separate lists depending on the dictionary
            if(len(hands_dict) == 1):
                screen_width = hands_dict[0]['screen_width']
                screen_height = hands_dict[0]['screen_height']
                if(hands_dict[0]['hand_class'] == 'Right'):
                    coordinates_right = hands_dict[0]['pos']
                if(hands_dict[0]['hand_class'] == 'Left'):
                    coordinates_left = hands_dict[0]['pos']
            if(len(hands_dict) == 2):
                screen_width = hands_dict[0]['screen_width']
                screen_height = hands_dict[0]['screen_height']
                if(hands_dict[0]['hand_class'] == 'Left' and hands_dict[1]['hand_class'] == 'Right'):
                    coordinates_left = hands_dict[0]['pos']
                    coordinates_right = hands_dict[1]['pos']
                if(hands_dict[0]['hand_class'] == 'Right' and hands_dict[1]['hand_class'] == 'Left'):
                    coordinates_right = hands_dict[0]['pos']
                    coordinates_left = hands_dict[1]['pos']
                
        csv_str = generate_cs_str(id,frameNr,screen_width,screen_height,coordinates_left,coordinates_right,movement)
            
        
        with open("khaledAzzab_dataset.csv", "a", newline="") as f:
            f.write(f'{csv_str}\n')

        cv2.imwrite(f'{dirName}/frame_{frameNr}.jpg',img)
        frameNr = frameNr+1

    capture.release()
    print(f"DONE : {fileName}")

In [19]:
#fileName = video_dirs['correct'][0]
FRAMESDIR = 'dataset_frames2'

count_videos = 0
try:
    os.mkdir(FRAMESDIR)
except Exception as e:
    pass

for movement in video_dirs:
    count_videos_per_movement = 0
    frames_dir_movement = FRAMESDIR + '/' + movement
    try:
        os.mkdir(frames_dir_movement)
    except Exception as e:
        pass
    #print(video_dirs[movement])
    for path in video_dirs[movement]:
        #print(path)
        #increment needs to be in the inner loop
        try:
            #print(frames_dir_movement + '/'+str(count_videos))
            os.mkdir(frames_dir_movement + '/'+str(count_videos))
        except Exception as e:
            pass
        #dir ill save in,path to video
        
        extract_frames(True,frames_dir_movement + '/'+str(count_videos),video_dirs[movement][count_videos_per_movement],count_videos,movement)
        count_videos += 1
        count_videos_per_movement += 1
        print(count_videos)
        
        


DONE : dataset2\correct\VID_20220218_163325.mp4
1
DONE : dataset2\correct\VID_20220218_163338.mp4
2
DONE : dataset2\correct\VID_20220218_163406.mp4
3
DONE : dataset2\correct\VID_20220218_163418.mp4
4
DONE : dataset2\correct\VID_20220218_163436.mp4
5
DONE : dataset2\correct\VID_20220218_163504.mp4
6
DONE : dataset2\correct\VID_20220218_163517.mp4
7
DONE : dataset2\correct\VID_20220218_163532.mp4
8
DONE : dataset2\correct\VID_20220218_163543.mp4
9
DONE : dataset2\correct\VID_20220218_163552.mp4
10
DONE : dataset2\correct\VID_20220218_163611.mp4
11
DONE : dataset2\correct\VID_20220218_163622.mp4
12
DONE : dataset2\correct\VID_20220218_163627.mp4
13
DONE : dataset2\correct\VID_20220218_163644.mp4
14
DONE : dataset2\wrong_placement\VID_20220218_164101.mp4
15
DONE : dataset2\wrong_placement\VID_20220218_164141.mp4
16
DONE : dataset2\wrong_placement\VID_20220218_164207.mp4
17
DONE : dataset2\wrong_placement\VID_20220218_164309.mp4
18
DONE : dataset2\wrong_placement\VID_20220218_164339.mp4
19
